# **Final Project Task 1 - Census Data Preprocess**

Requirements

- Target variable specification:
    - The target variable for this project is hours-per-week. 
    - Ensure all preprocessing steps are designed to support regression analysis on this target variable.
- Encode data  **3p**
- Handle missing values if any **1p**
- Correct errors, inconsistencies, remove duplicates if any **1p**
- Outlier detection and treatment if any **1p**
- Normalization / Standardization if necesarry **1p**
- Feature engineering **3p**
- Train test split, save it.
- Others?


Deliverable:

- Notebook code with no errors.
- Preprocessed data as csv.

In [13]:
import pandas as pd

In [8]:
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

data = pd.read_csv(data_url, header=None, names=columns, na_values=" ?", skipinitialspace=True)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Setup + încarcare datasetului

In [9]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

RANDOM_STATE = 42

adult = fetch_openml("adult", version=2, as_frame=True)
df = adult.frame.copy()

df.shape, df.head()

((48842, 15),
    age  workclass  fnlwgt     education  education-num      marital-status  \
 0   25    Private  226802          11th              7       Never-married   
 1   38    Private   89814       HS-grad              9  Married-civ-spouse   
 2   28  Local-gov  336951    Assoc-acdm             12  Married-civ-spouse   
 3   44    Private  160323  Some-college             10  Married-civ-spouse   
 4   18        NaN  103497  Some-college             10       Never-married   
 
           occupation relationship   race     sex  capital-gain  capital-loss  \
 0  Machine-op-inspct    Own-child  Black    Male             0             0   
 1    Farming-fishing      Husband  White    Male             0             0   
 2    Protective-serv      Husband  White    Male             0             0   
 3  Machine-op-inspct      Husband  Black    Male          7688             0   
 4                NaN    Own-child  White  Female             0             0   
 
    hours-per-week nat

**Observati**: Datasetul Adult Census a fost încărcat și conține 48.842 de observații și 15 variabile, care descriu caracteristici demografice, educaționale și profesionale ale indivizilor.
Variabila țintă a proiectului este `hours-per-week`, o variabilă numerică continuă. 
Prin urmare, problema abordată este una de regresie, iar toate etapele de preprocesare sunt realizate pentru a susține acest tip de analiză.

### Alegerea Variabilei

In [10]:
target = "hours-per-week"

# asigurare tip numeric
df[target] = pd.to_numeric(df[target], errors="coerce")

X = df.drop(columns=[target])
y = df[target]

y.describe()

count    48842.000000
mean        40.422382
std         12.391444
min          1.000000
25%         40.000000
50%         40.000000
75%         45.000000
max         99.000000
Name: hours-per-week, dtype: float64

**Observati**: Variabila țintă aleasă pentru acest proiect este `hours-per-week`, care reprezintă numărul de ore lucrate pe săptămână de către fiecare individ.

- este de tip numeric continuu, ceea ce face ca problema abordată să fie una de regresie. 
- a fost convertită explicit la tip numeric, iar valorile nevalide au fost tratate ca valori lipsă.

Analiza statistică descriptivă arată că majoritatea valorilor sunt concentrate în jurul a 40 de ore pe săptămână, cu o variabilitate moderată și câteva valori extreme, care vor fi analizate în etapele ulterioare de preprocesare.


### Corectarea erorilor, inconsistențelor și eliminarea duplicatelor

In [11]:
df = data.copy()

# Standardize categorical strings (extra safety even if you used skipinitialspace=True)
obj_cols = df.select_dtypes(include="object").columns
df[obj_cols] = df[obj_cols].apply(lambda s: s.str.strip())

# normalize internal multiple spaces (rare, but safe)
df[obj_cols] = df[obj_cols].replace(r"\s+", " ", regex=True)

# Check duplicates
n_dup = df.duplicated().sum()
print(f"Duplicate rows found: {n_dup}")

if n_dup > 0:
    df = df.drop_duplicates()
    print(f"Duplicates removed. New shape: {df.shape}")
else:
    print("No duplicates to remove.")

# Missing values per column (shows that '?' were converted to NaN)
missing_counts = df.isna().sum().sort_values(ascending=False)
print("\nMissing values per column (top):")
print(missing_counts[missing_counts > 0].head(10))

# Update the main dataframe
data = df

data.shape, data.head()

Duplicate rows found: 24
Duplicates removed. New shape: (32537, 15)

Missing values per column (top):
Series([], dtype: int64)


((32537, 15),
    age         workclass  fnlwgt  education  education-num  \
 0   39         State-gov   77516  Bachelors             13   
 1   50  Self-emp-not-inc   83311  Bachelors             13   
 2   38           Private  215646    HS-grad              9   
 3   53           Private  234721       11th              7   
 4   28           Private  338409  Bachelors             13   
 
        marital-status         occupation   relationship   race     sex  \
 0       Never-married       Adm-clerical  Not-in-family  White    Male   
 1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
 2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
 3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
 4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   
 
    capital-gain  capital-loss  hours-per-week native-country income  
 0          2174             0              40  United-States  <=50K  
 1          

**Observati**: Au fost efectuate verificări de calitate asupra datasetului pentru a corecta eventualele inconsistențe și pentru a elimina datele redundante.

După curățare, datasetul nu mai conține duplicate, iar valorile lipsă sunt gestionate explicit sub formă de NaN, fiind pregătite pentru etapele ulterioare de preprocesare.

### Tratarea valorilor lipsă

In [14]:
# Analiza valorilor lipsă
missing_counts = data.isna().sum()
missing_percent = (missing_counts / len(data)) * 100

missing_summary = pd.DataFrame({
    "Missing values": missing_counts,
    "Percentage (%)": missing_percent
}).sort_values(by="Missing values", ascending=False)

missing_summary[missing_summary["Missing values"] > 0]

,Missing values,Percentage (%)


**Observati**: Analiza indică faptul că datasetul nu conține valori lipsă după etapa de curățare a datelor.

### Detectarea și tratarea outlierilor
(IQR Outliers + Capping)

In [15]:
df = data.copy()

# alegem doar coloanele numerice
target = "hours-per-week"
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# nu tratăm outlierii pe target (doar pe features)
feature_num_cols = [c for c in num_cols if c != target]

def iqr_bounds(series, factor=1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - factor * iqr
    upper = q3 + factor * iqr
    return lower, upper

# Raport outlieri înainte
outlier_report = []
bounds_dict = {}

for col in feature_num_cols:
    lower, upper = iqr_bounds(df[col], factor=1.5)
    bounds_dict[col] = (lower, upper)

    outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    outlier_report.append([col, outliers, round(outliers / len(df) * 100, 2), lower, upper])

outlier_report_df = pd.DataFrame(
    outlier_report,
    columns=["Column", "Outliers (count)", "Outliers (%)", "Lower bound", "Upper bound"]
).sort_values(by="Outliers (count)", ascending=False)

outlier_report_df.head(10)

,Column,Outliers (count),Outliers (%),Lower bound,Upper bound
3,capital-gain,2712,8.34,0.0,0.0
4,capital-loss,1519,4.67,0.0,0.0
2,education-num,1193,3.67,4.5,16.5
1,fnlwgt,993,3.05,-60922.0,415742.0
0,age,142,0.44,-2.0,78.0


In [16]:
# Aplicăm capping (winsorization) doar pe features numerice
df_capped = df.copy()

for col, (lower, upper) in bounds_dict.items():
    df_capped[col] = df_capped[col].clip(lower, upper)

# Verificăm outlierii după capping (ar trebui să scadă mult / spre 0)
outlier_report_after = []

for col in feature_num_cols:
    lower, upper = bounds_dict[col]
    outliers_after = ((df_capped[col] < lower) | (df_capped[col] > upper)).sum()
    outlier_report_after.append([col, outliers_after])

outlier_after_df = pd.DataFrame(outlier_report_after, columns=["Column", "Outliers after capping"])
outlier_after_df.sort_values(by="Outliers after capping", ascending=False).head(10)

,Column,Outliers after capping
0,age,0
1,fnlwgt,0
2,education-num,0
3,capital-gain,0
4,capital-loss,0


**Observati**: Analiza outlierilor a fost realizată utilizând regula IQR (1.5 × IQR). 
Rezultatele indică prezența unui număr semnificativ de valori extreme în variabile precum `capital-gain`, `capital-loss` și `education-num`, în timp ce alte variabile prezintă un număr redus de outlieri.

Pentru a reduce influența acestor valori extreme asupra modelelor de regresie, a fost aplicat un tratament de tip capping (winsorization), prin limitarea valorilor la intervalele inferioare și superioare definite de IQR.

După aplicarea capping-ului, nu mai sunt detectați outlieri în variabilele numerice analizate, iar datasetul este pregătit pentru etapele următoare de preprocesare.


Variabila țintă `hours-per-week` nu a fost modificată în această etapă pentru a păstra interpretabilitatea valorilor reale.


In [17]:
data = df_capped

data.shape, data.head()

((32537, 15),
    age         workclass  fnlwgt  education  education-num  \
 0   39         State-gov   77516  Bachelors           13.0   
 1   50  Self-emp-not-inc   83311  Bachelors           13.0   
 2   38           Private  215646    HS-grad            9.0   
 3   53           Private  234721       11th            7.0   
 4   28           Private  338409  Bachelors           13.0   
 
        marital-status         occupation   relationship   race     sex  \
 0       Never-married       Adm-clerical  Not-in-family  White    Male   
 1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
 2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
 3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
 4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   
 
    capital-gain  capital-loss  hours-per-week native-country income  
 0             0             0              40  United-States  <=50K  
 1          

### Standardizare+ Nornalizare

In [19]:

RANDOM_STATE = 42
target = "hours-per-week"

X = data.drop(columns=[target])
y = data[target]

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()

num_cols, cat_cols[:5]

(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss'],
 ['workclass', 'education', 'marital-status', 'occupation', 'relationship'])

In [20]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ],
    remainder="drop"
)

In [21]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

X_train_processed.shape, X_test_processed.shape


((26029, 109), (6508, 109))

**Observati**: Pentru a susține analiza de regresie, variabilele numerice au fost standardizate folosind `StandardScaler`, astfel încât acestea să aibă medie zero și deviație standard egală cu unu.

Standardizarea a fost realizată exclusiv pe setul de antrenare, iar parametrii obținuți au fost aplicați ulterior asupra setului de test, prevenind astfel scurgerea de informație (data leakage).

Variabilele categoriale au fost transformate prin One-Hot Encoding, rezultând un set de date numeric complet, adecvat pentru antrenarea modelelor de regresie. 
După aplicarea transformărilor, setul de antrenare conține 26.029 observații și 109 variabile, iar setul de test 6.508 observații, păstrând aceeași structură a caracteristicilor.


### Feature Engineering

In [ ]:
def add_features(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()

    # 1) Capital net + log transforms
    if "capital-gain" in df.columns and "capital-loss" in df.columns:
        df["capital_net"] = df["capital-gain"].fillna(0) - df["capital-loss"].fillna(0)
        df["capital_gain_log"] = np.log1p(df["capital-gain"].fillna(0))
        df["capital_loss_log"] = np.log1p(df["capital-loss"].fillna(0))
        df["has_capital"] = ((df["capital-gain"].fillna(0) > 0) | (df["capital-loss"].fillna(0) > 0)).astype(int)

    # 2) Age bin 
    if "age" in df.columns:
        df["age_bin"] = pd.cut(
            df["age"],
            bins=[0, 25, 35, 45, 60, 100],
            labels=["<25", "25-34", "35-44", "45-59", "60+"],
            include_lowest=True
        )

    # 3) Married flag 
    if "marital-status" in df.columns:
        ms = df["marital-status"].astype(str)
        df["is_married"] = ms.str.contains("Married", na=False).astype(int)

    # 4) Work intensity (ore / varsta) - numeric
    if "hours-per-week" in df.columns and "age" in df.columns:
        pass

    # 5) Education
    if "education-num" in df.columns:
        df["high_edu"] = (df["education-num"] >= 13).astype(int) 

    # 6) Relationship simplificat
    if "relationship" in df.columns:
        rel = df["relationship"].astype(str)
        df["is_head_family"] = rel.isin(["Husband", "Wife"]).astype(int)

    return df

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42
target = "hours-per-week"

X = data.drop(columns=[target])
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

# feature engineering ca prim pas în pipeline
feat_eng = FunctionTransformer(add_features, validate=False)

# Pentru a afla coloanele după feature engineering, aplicăm pe train (doar pentru listă)
X_train_fe = add_features(X_train)

num_cols = X_train_fe.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train_fe.select_dtypes(include=["object", "category"]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop"
)

# feature engineering -> preprocess (scaler + onehot)
full_pipeline = Pipeline(steps=[
    ("feature_engineering", feat_eng),
    ("preprocess", preprocessor)
])

X_train_processed = full_pipeline.fit_transform(X_train)
X_test_processed = full_pipeline.transform(X_test)

X_train_processed.shape, X_test_processed.shape

((26029, 121), (6508, 121))

In [27]:
# Nume de coloane după OneHot
ohe = full_pipeline.named_steps["preprocess"].named_transformers_["cat"].named_steps["onehot"]
feature_names_cat = ohe.get_feature_names_out(cat_cols)
feature_names = np.concatenate([num_cols, feature_names_cat])

# sparse -> dense dacă e nevoie
Xtr = X_train_processed.toarray() if hasattr(X_train_processed, "toarray") else X_train_processed
Xte = X_test_processed.toarray() if hasattr(X_test_processed, "toarray") else X_test_processed

X_train_df = pd.DataFrame(Xtr, columns=feature_names, index=X_train.index)
X_test_df  = pd.DataFrame(Xte, columns=feature_names, index=X_test.index)

train_final = X_train_df.copy()
train_final[target] = y_train.values

test_final = X_test_df.copy()
test_final[target] = y_test.values

train_final.to_csv("train_preprocessed.csv", index=False)
test_final.to_csv("test_preprocessed.csv", index=False)

train_final.shape, test_final.shape


((26029, 122), (6508, 122))

**Observati**: Aplicarea etapelor de feature engineering și a transformărilor ulterioare a dus la creșterea dimensionalității datasetului.

După preprocesare, setul de antrenare conține 26.029 observații și 121 de variabile explicative, iar setul de test 6.508 observații și aceeași structură de 121 de variabile. 
După atașarea variabilei țintă `hours-per-week`, seturile finale conțin 122 de coloane.

Creșterea numărului de variabile este determinată de:
- crearea de variabile derivate (ex. `capital_net`, `has_capital`, `high_edu`);
- discretizarea vârstei (`age_bin`);
- aplicarea One-Hot Encoding asupra variabilelor categoriale, unde fiecare categorie este reprezentată printr-o variabilă binară.

Aceste transformări permit modelului de regresie să capteze relații mai complexe din date, menținând în același timp consistența între setul de antrenare și cel de test.

Seturile de date preprocesate au fost salvate sub formă de fișiere CSV separate pentru antrenare și testare, conținând exclusiv variabile numerice și variabila țintă, fiind astfel pregătite pentru etapele de modelare.
